In [1]:
import numpy as np

import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['figure.dpi'] = 100

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import string
import mplcursors
import collections
import sklearn.cluster
import os
from tqdm import tqdm_notebook
#%matplotlib inline
%matplotlib notebook
#%matplotlib notebook

In [141]:
# NEW IMPLEMENTATION

import math

def get_intersections(hist1, hist2):
    len1 = len(hist1)
    len2 = len(hist2)
    intersections = []
    i = 0
    j = 0

    while i < len1 and j < len2:
        if hist1[i][0] < hist2[j][0]:
            i += 1
        elif hist1[i][0] == hist2[j][0]:
            intersections.append((hist1[i][0], hist1[i][1], hist2[j][1]))
            i += 1
            j += 1
        elif hist1[i][0] > hist2[j][0]:
            j += 1
            
    return intersections

def get_union_inds(hist1, hist2):
    len1 = len(hist1)
    len2 = len(hist2)
    union = []
    i = 0
    j = 0

    while i < len1 or j < len2:
        if i >= len1:
            union.append(hist2[j][0])
            j += 1
        elif j >= len2:
            union.append(hist1[i][0])
            i += 1
        elif hist1[i][0] < hist2[j][0]:
            union.append(hist1[i][0])
            i += 1
        elif hist1[i][0] == hist2[j][0]:
            union.append(hist1[i][0])
            i += 1
            j += 1
        elif hist1[i][0] > hist2[j][0]:
            union.append(hist2[j][0])
            j += 1
            
    return union


def jaccard_metric(hist1, hist2, hist_len=None):
    intersections = get_intersections(hist1, hist2)

    metric = 0

    for ind, v1, v2 in intersections:
        mx = max(v1, v2)
        mn = min(v1, v2)
        metric += mn / mx

    len_intersections = len(intersections)

    if (len_intersections != 0):
        metric = metric / len(intersections)
    else:
        metric = 0

    return 1 - metric

def canberra_metric(hist1, hist2, hist_len=None):
    metric = 0
    i = 0
    j = 0
    n = len(hist1)
    m = len(hist2)
    
    union_len = 0
    
    while i < n or j < m:
        if i >= n:
            metric += abs(hist2[j][1]) / abs(hist2[j][1]) # 1
            j += 1
        elif j >= m:
            metric += abs(hist1[i][1]) / abs(hist1[i][1]) # 1
            i += 1
        elif hist1[i][0] == hist2[j][0]:
            metric += abs(hist1[i][1] - hist2[j][1]) / (abs(hist1[i][1]) + abs(hist2[j][1]))
            i += 1
            j += 1
        elif hist1[i][0] < hist2[j][0]:
            metric += abs(hist1[i][1]) / abs(hist1[i][1]) # 1
            i += 1
        elif hist1[i][0] > hist2[j][0]:
            metric += abs(hist2[j][1]) / abs(hist2[j][1]) # 1
            j += 1
        
        #union_len += 1
            
    union_len = len(get_union_inds(hist1, hist2))
    if union_len == 0:
        return 1
    
    return metric / union_len


def canberra_metric_optimized(hist1, hist2, hist_len=None):
    metric = 0
    i = 0
    j = 0
    n = len(hist1)
    m = len(hist2)
    
    union_len = 0
    
    while i < n or j < m:
        if i >= n:
            metric += 1.0
            j += 1
        elif j >= m:
            metric += 1.0
            i += 1
        elif hist1[i][0] == hist2[j][0]:
            metric += abs(hist1[i][1] - hist2[j][1]) / (abs(hist1[i][1]) + abs(hist2[j][1]))
            i += 1
            j += 1
        elif hist1[i][0] < hist2[j][0]:
            metric += 1.0
            i += 1
        elif hist1[i][0] > hist2[j][0]:
            metric += 1.0
            j += 1
        
        union_len += 1
            
    #union_len = len(get_union_inds(hist1, hist2))
    if union_len == 0:
        return 1
    
    return metric / union_len



'''
def pearsons_correlation(hist1, hist2, hist_len):
    #hist_len = hist_len_3gram
    union_len = len(get_union_inds(hist1, hist2))
    
    top = 0
    left = 0
    right = 0
    
    n = len(hist1)
    m = len(hist2)
    
    i = 0
    j = 0
    
    while i < n or j < m:
        if i >= n:
            top += (- 1/hist_len) * (hist2[j][1] - 1/hist_len)
            left += (- 1/hist_len) ** 2
            right += (hist2[j][1] - 1/hist_len) ** 2
            j += 1
        elif j >= m:
            top += (hist1[i][1] - 1/hist_len) * (- 1/hist_len)
            left += (hist1[i][1] - 1/hist_len)  ** 2
            right += (- 1/hist_len) ** 2
            i += 1
        elif hist1[i][0] == hist2[j][0]:
            top += (hist1[i][1] - 1/hist_len) * (hist2[j][1] - 1/hist_len)
            left += (hist1[i][1] - 1/hist_len)  ** 2
            right += (hist2[j][1] - 1/hist_len) ** 2
            i += 1
            j += 1
        elif hist1[i][0] < hist2[j][0]:
            top += (hist1[i][1] - 1/hist_len) * (- 1/hist_len)
            left += (hist1[i][1] - 1/hist_len)  ** 2
            right += (- 1/hist_len) ** 2
            i += 1
        elif hist1[i][0] > hist2[j][0]:
            top += (- 1/hist_len) * (hist2[j][1] - 1/hist_len)
            left += (- 1/hist_len) ** 2
            right += (hist2[j][1] - 1/hist_len) ** 2
            j += 1

    bottom = math.sqrt(left * right)
    return 1 - top / bottom
'''


def cos_distance(hist1, hist2, hist_len=None):
    intersections = get_intersections(hist1, hist2)

    top = 0

    for ind, v1, v2 in intersections:
        top += v1 * v2
        
    bottom1 = (sum([pair[1] ** 2 for pair in hist1]))
    bottom2 = (sum([pair[1] ** 2 for pair in hist2]))
    
    #print(top)
    #print(bottom1)
    #print(bottom2)
    
    return 1 - abs(top / np.sqrt(bottom1 * bottom2))
    

def pearsons_correlation_mean(hist1, hist2, hist_len):
    union_len = len(get_union_inds(hist1, hist2))
    
    top = 0
    left = 0
    right = 0
    
    n = len(hist1)
    m = len(hist2)
    
    mean1 = sum([pair[1] for pair in hist1]) / hist_len
    mean2 = sum([pair[1] for pair in hist2]) / hist_len
    
    i = 0
    j = 0
    
    while i < n or j < m:
        if i >= n:
            top += (- mean1) * (hist2[j][1] - mean2)
            left += (- mean1) ** 2
            right += (hist2[j][1] - mean2) ** 2
            j += 1
        elif j >= m:
            top += (hist1[i][1] - mean1) * (- mean2)
            left += (hist1[i][1] - mean1)  ** 2
            right += (- mean2) ** 2
            i += 1
        elif hist1[i][0] == hist2[j][0]:
            top += (hist1[i][1] - mean1) * (hist2[j][1] - mean2)
            left += (hist1[i][1] - mean1)  ** 2
            right += (hist2[j][1] - mean2) ** 2
            i += 1
            j += 1
        elif hist1[i][0] < hist2[j][0]:
            top += (hist1[i][1] - mean1) * (- mean2)
            left += (hist1[i][1] - mean1)  ** 2
            right += (- mean2) ** 2
            i += 1
        elif hist1[i][0] > hist2[j][0]:
            top += (- mean1) * (hist2[j][1] - mean2)
            left += (- mean1) ** 2
            right += (hist2[j][1] - mean2) ** 2
            j += 1

    bottom = math.sqrt(left * right)
    return round(1 - top / bottom, 15)
    

def get_dists(hists, dist_metric, hist_len):
    #hist_len = len(hists[0])
    
    #n = len(hists)
    dists = []
    
    for hist1 in tqdm_notebook(hists):
        cur_dists = []

        for hist2 in hists:
            distance = dist_metric(hist1, hist2, hist_len)
            
            cur_dists.append(distance)

        dists.append(cur_dists)
        
    return dists


def get_dists_optimized(hists, dist_metric, hist_len):
    #hist_len = len(hists[0])
    
    n = len(hists)
    #dists = [[0.0 for i in range(n)] for j in range(n)]
    dists = np.zeros((n, n))
    
    for i in tqdm_notebook(range(n)):
        for j in range(n):
            if i <= j:
                break
                
            distance = dist_metric(hists[i], hists[j], hist_len)
            dists[i][j] = distance
            dists[j][i] = distance
        
    return dists

In [3]:
RESULTS_LASE = "/Volumes/Seagate/Alina/result_for_lase_treat-false"

In [4]:
all_dirs = [os.fsdecode(el) for el in os.listdir(RESULTS_LASE) 
               if os.path.isdir(RESULTS_LASE + "/" + os.fsdecode(el))]
all_dirs = sorted(all_dirs, key=lambda s: int(s.split(" ")[0])) 
len(all_dirs)

149

In [5]:
LASE_DATASET_PATH = "/Users/aliscafo/Documents/ALINA/WORK/SPbAU/thesis/CodeDiffEditScripts/LaseDataset"

label_to_changes = collections.defaultdict(list)
from_change_to_label = collections.defaultdict(list)

for double_num in tqdm_notebook(all_dirs):
    change_id = double_num.split(" ")[0]
    label_file = open(LASE_DATASET_PATH + "/" + change_id + "/" + "label")
    label = int(label_file.read())
    
    label_to_changes[label].append(change_id)
    from_change_to_label[change_id].append(label)

    label_file.close()
    

In [6]:
print(len(from_change_to_label))
print("NUM CLASSES:", len(label_to_changes))

149
NUM CLASSES: 24


In [7]:
ids_per_label = []
lines_poses = []
cur_sum = 0
all_labels = []

for label in label_to_changes:
    all_labels.append(label)
    for change in label_to_changes[label]:
        ids_per_label.append(change)
        cur_sum += 1
    lines_poses.append(cur_sum)

In [8]:
for i in range(len(ids_per_label)):
    print("On index", i, ids_per_label[i])

On index 0 1
On index 1 2
On index 2 3
On index 3 4
On index 4 5
On index 5 6
On index 6 7
On index 7 8
On index 8 9
On index 9 10
On index 10 11
On index 11 12
On index 12 13
On index 13 14
On index 14 15
On index 15 16
On index 16 17
On index 17 18
On index 18 19
On index 19 20
On index 20 21
On index 21 22
On index 22 23
On index 23 24
On index 24 25
On index 25 26
On index 26 27
On index 27 28
On index 28 29
On index 29 30
On index 30 31
On index 31 32
On index 32 33
On index 33 34
On index 34 35
On index 35 36
On index 36 37
On index 37 38
On index 38 39
On index 39 40
On index 40 41
On index 41 42
On index 42 43
On index 43 44
On index 44 45
On index 45 46
On index 46 47
On index 47 48
On index 48 49
On index 49 50
On index 50 51
On index 51 52
On index 52 53
On index 53 54
On index 54 55
On index 55 56
On index 56 57
On index 57 58
On index 58 59
On index 59 60
On index 60 61
On index 61 62
On index 62 63
On index 63 64
On index 64 65
On index 65 66
On index 66 67
On index 67 68

In [9]:
for i in range(len(lines_poses)):
    if i == 0:
        print("From 0 to", lines_poses[i], all_labels[i])
    else:
        print("From", lines_poses[i - 1], "to", lines_poses[i], all_labels[i])

From 0 to 4 1
From 4 to 20 2
From 20 to 24 3
From 24 to 30 4
From 30 to 42 5
From 42 to 45 6
From 45 to 52 7
From 52 to 56 8
From 56 to 60 9
From 60 to 63 10
From 63 to 66 11
From 66 to 75 12
From 75 to 81 13
From 81 to 84 14
From 84 to 87 15
From 87 to 96 16
From 96 to 100 17
From 100 to 105 18
From 105 to 111 19
From 111 to 116 20
From 116 to 126 21
From 126 to 129 22
From 129 to 134 23
From 134 to 149 24


In [27]:
def print_clustering_results_tufano_unique(clustering, dists, unique_label_to_changes, 
                                    from_change_to_unique_label, ids_per_unique_label, 
                                    without_outliers,
                                    to_print=True):
    clusters = collections.defaultdict(list)
    clusters_to_ids = collections.defaultdict(list)
    outliers = []
    num_changes = len(clustering.labels_)
    
    for i in range(len(clustering.labels_)):
        label = clustering.labels_[i]
        change = ids_per_unique_label[i]
        if label == -1:
            outliers.append(change)
        else:
            clusters[label].append(change)
            clusters_to_ids[label].append(i)
    
    # For Agglomerative
    clusters_list = list(clusters_to_ids.keys())
    for label in clusters_list:
        if len(clusters_to_ids[label]) == 1:
            outliers.append(clusters[label][0])
            clustering.labels_[clusters_to_ids[label][0]] = -1
            clusters_to_ids.pop(label, None)
            clusters.pop(label, None)
            
    num_inliers = num_changes - len(outliers)   
            
    cohesion = 0
    separation = 0
    g1 = 0
    MAX_DIST = np.array(dists).max()
    #print("MAX_DIST", MAX_DIST)
    for i in clusters_to_ids.keys():
        coef = 0
        #print("!!!", len(clusters_to_ids[i]))
        for xi in range(len(clusters_to_ids[i])):
            for yi in range(len(clusters_to_ids[i])):
                if xi < yi:
                    x = clusters_to_ids[i][xi]
                    y = clusters_to_ids[i][yi]
                    #print("dists[x][y]", dists[x][y])
                    coef += MAX_DIST - dists[x][y]
                    
        cohesion += (1 / len(clusters_to_ids[i])) * coef
        coef_for_sep = len(clusters_to_ids[i]) / np.sqrt(coef)
        coef = 1 / coef
        
        summ = 0
        for j in clusters_to_ids.keys():
            if i == j:
                continue
                
            for xi in range(len(clusters_to_ids[i])):
                for yi in range(len(clusters_to_ids[j])):
                    x = clusters_to_ids[i][xi]
                    y = clusters_to_ids[j][yi]
                    summ += MAX_DIST - dists[x][y]
        
        g1 += coef * summ
        separation += coef_for_sep * summ
        
        
    #final_init_labels = dict()
        
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for i in range(len(clustering.labels_)):
        change_i = ids_per_unique_label[i]
        init_label_i = from_change_to_unique_label[change_i][0]
        after_label_i = clustering.labels_[i]
        
        for j in range(len(clustering.labels_)):
            change_j = ids_per_unique_label[j]
            init_label_j = from_change_to_unique_label[change_j][0]
            after_label_j = clustering.labels_[j]
            
            if i >= j:
                continue
                
            if (without_outliers and (after_label_i == -1 or after_label_j == -1)):
                continue
            
            if init_label_i == init_label_j:
                if after_label_i == after_label_j:
                    tp += 1
                else:
                    fp += 1
            else:
                if after_label_i == after_label_j:
                    tn += 1
                else:
                    fn += 1
                    
                    
    
    confusion_mtx = []
    for i in clusters_to_ids.keys():
        mtx_row = collections.defaultdict(int)
        
        for change_i in clusters_to_ids[i]:
            change = ids_per_unique_label[change_i]
            init_label = from_change_to_unique_label[change][0]
            mtx_row[init_label] += 1
        
        confusion_mtx.append(mtx_row)
        
    ENTROPY = 0
    PURITY = 0
    entropies = []
    purities = []
    Fs = collections.defaultdict(list)
    for i in range(len(clusters_to_ids.keys())):
        entropy = 0
        purity = 0
        
        clustering_label = list(clusters_to_ids.keys())[i]
        
        for init_lbl in unique_label_to_changes:
            pij = confusion_mtx[i][init_lbl] / len(clusters_to_ids[clustering_label])
            precision = pij
            recall = confusion_mtx[i][init_lbl] / len(unique_label_to_changes[init_lbl])
            
            if pij != 0:
                entropy += - pij * np.log2(pij)
            purity = max(purity, pij)
            
            fij = 0
            if precision != 0 or recall != 0:
                fij = (2 * precision * recall) / (precision + recall)
            Fs[init_lbl].append(fij)
        
        ENTROPY += entropy * len(clusters_to_ids[clustering_label]) / num_inliers
        entropies.append(round(entropy, 3))
        
        PURITY += purity * len(clusters_to_ids[clustering_label]) / num_inliers
        purities.append(round(purity, 3))
        
    Fmeasure = 0
    for init_lbl in unique_label_to_changes:
        Fmeasure += max(Fs[init_lbl]) * len(unique_label_to_changes[init_lbl]) / num_changes
            
    
    rand = round(100 * (tp + fn) / (tp + tn + fp + fn), 3)
    jaccard_index = round(100 * tp / (tp + tn + fp), 3)
    outl_percent = round(100 * len(outliers) / num_changes, 3)
    
    
    if (to_print):
        '''
        print("CLUSTERS:\n")

        for i in clusters.keys():
            for change in clusters[i]:
                print(change, from_change_to_unique_label[change])
            print("\n")
        '''
        
        print("Number of clusters:", len(clusters))
        print("Number of outliers:", str(len(outliers)) + "/" + str(num_changes), "(" + str(outl_percent) + "%)")
        
        print()
        print("Cohesion =", str(cohesion))
        print("Separation =", str(separation))
        print("G1 =", str(g1))
        
        print()
        print("Entropy =", str(round(ENTROPY, 6)))
        print("Purity =", str(round(PURITY, 6)))
        #pairs = [(entropies[i], purities[i]) for i in range(len(entropies))]
        #print("(entropy, purity):", pairs)
        print("F-measure =", round(Fmeasure, 6))
        
        print()
        print("Rand =", str(rand) + "%")
        print("Jaccard Index =", str(jaccard_index) + "%")       
    
    return rand, Fmeasure
    
    

In [208]:
def find_eps_with_brute_force_universal(dists, label_to_changes, from_change_to_label, 
                                        ids_per_label, agglomerative=False, linkage='single', with_step=None):
    if with_step is None:
        all_dists = np.unique(np.array(dists).flatten())
    else:
        all_dists = np.arange(0.0, max(1.1, np.array(dists).flatten().max()), with_step)
        
    if agglomerative is True:
        print("Agglomerative Clustering")
    else:
        print("DBSCAN")

    max_rand = 0
    best_eps = -1
    max_fm = 0

    for eps in tqdm_notebook(all_dists):
        if eps <= 0.0:
            continue
            
        if agglomerative is True:
            tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=linkage, 
                                                            compute_full_tree=True,
                                                            distance_threshold=eps).fit(dists)
        else:
            tufano_clustering = sklearn.cluster.DBSCAN(eps=eps, 
                                                   min_samples=2, 
                                                   metric='precomputed').fit(dists)
        
        rand, fm = print_clustering_results_tufano_unique(tufano_clustering, 
                                               dists,
                                               label_to_changes, 
                                               from_change_to_label, 
                                               ids_per_label,
                                               to_print=False, 
                                               without_outliers=True)

        if max_fm < fm:
            max_fm = fm
            best_eps = eps

    print("Best eps:", best_eps)
    print("Best F-measure:", max_fm)
    
    return best_eps

In [12]:
def get_lase_hists(hists_path, gram_path):
    hists = []

    for double_num in tqdm_notebook(ids_per_label):
        hist_path1 = hists_path + "/" + double_num + " " + double_num + "/" + gram_path + "/sampleChange1" + "_hist.txt"
        
        if os.path.exists(hist_path1):
            hist_file1 = open(hist_path1, "r")
            lines = hist_file1.read().split("\n")
            hist_data1 = [(int(line.split(" ")[0]), int(line.split(" ")[1])) for line in lines if line != '']
            hists.append(hist_data1)
            hist_file1.close()
        else:
            hists.append([])
        
    return hists

def get_lase_hists_len(hists_path, gram_path):
    hists_len = None
    
    es_path = hists_path + "/" + "edit_scripts_" + gram_path + "s_mapped.txt"
    es_file = open(es_path, "r")
    es_lines = es_file.read().split("\n")
    hists_len = int(es_lines[-2].split(" ")[0][:-1]) + 1
    es_file.close()

    print(hists_len)
    
    return hists_len

In [86]:
#RESULTS_LASE = "/Volumes/Seagate/Alina/result_for_lase_treat-false"

#RESULTS_LASE = "/Volumes/Seagate/Alina/result_for_lase_with_context_concat"

RESULTS_LASE = "/Volumes/Seagate/Alina/result_for_lase_wo_context_concat"

In [87]:
hists_lase_1gram = get_lase_hists(RESULTS_LASE, "1gram")
hists_lase_2gram = get_lase_hists(RESULTS_LASE, "2gram")
hists_lase_3gram = get_lase_hists(RESULTS_LASE, "3gram")
hists_lase_4gram = get_lase_hists(RESULTS_LASE, "4gram")
hists_lase_5gram = get_lase_hists(RESULTS_LASE, "5gram")
hists_lase_6gram = get_lase_hists(RESULTS_LASE, "6gram")
hists_lase_7gram = get_lase_hists(RESULTS_LASE, "7gram")
hists_lase_8gram = get_lase_hists(RESULTS_LASE, "8gram")
hists_lase_9gram = get_lase_hists(RESULTS_LASE, "9gram")
hists_lase_10gram = get_lase_hists(RESULTS_LASE, "10gram")

In [14]:
'''
hist_len_1gram_lase = get_lase_hists_len(RESULTS_LASE, "1gram")
hist_len_2gram_lase = get_lase_hists_len(RESULTS_LASE, "2gram")
hist_len_3gram_lase = get_lase_hists_len(RESULTS_LASE, "3gram")
hist_len_4gram_lase = get_lase_hists_len(RESULTS_LASE, "4gram")
hist_len_5gram_lase = get_lase_hists_len(RESULTS_LASE, "5gram")
hist_len_6gram_lase = get_lase_hists_len(RESULTS_LASE, "6gram")
hist_len_7gram_lase = get_lase_hists_len(RESULTS_LASE, "7gram")
hist_len_8gram_lase = get_lase_hists_len(RESULTS_LASE, "8gram")
hist_len_9gram_lase = get_lase_hists_len(RESULTS_LASE, "9gram")
hist_len_10gram_lase = get_lase_hists_len(RESULTS_LASE, "10gram")
'''

409
1199
1726
2009
2167
2258
2313
2342
2358
2370


In [62]:
'''
hist_len_1gram_lase = get_lase_hists_len(RESULTS_LASE, "1gram")
hist_len_2gram_lase = get_lase_hists_len(RESULTS_LASE, "2gram")
hist_len_3gram_lase = get_lase_hists_len(RESULTS_LASE, "3gram")
hist_len_4gram_lase = get_lase_hists_len(RESULTS_LASE, "4gram")
hist_len_5gram_lase = get_lase_hists_len(RESULTS_LASE, "5gram")
hist_len_6gram_lase = get_lase_hists_len(RESULTS_LASE, "6gram")
hist_len_7gram_lase = get_lase_hists_len(RESULTS_LASE, "7gram")
hist_len_8gram_lase = get_lase_hists_len(RESULTS_LASE, "8gram")
hist_len_9gram_lase = get_lase_hists_len(RESULTS_LASE, "9gram")
hist_len_10gram_lase = get_lase_hists_len(RESULTS_LASE, "10gram")
'''

423
1221
1735
2016
2170
2259
2313
2342
2358
2370


In [88]:
hist_len_1gram_lase = get_lase_hists_len(RESULTS_LASE, "1gram")
hist_len_2gram_lase = get_lase_hists_len(RESULTS_LASE, "2gram")
hist_len_3gram_lase = get_lase_hists_len(RESULTS_LASE, "3gram")
hist_len_4gram_lase = get_lase_hists_len(RESULTS_LASE, "4gram")
hist_len_5gram_lase = get_lase_hists_len(RESULTS_LASE, "5gram")
hist_len_6gram_lase = get_lase_hists_len(RESULTS_LASE, "6gram")
hist_len_7gram_lase = get_lase_hists_len(RESULTS_LASE, "7gram")
hist_len_8gram_lase = get_lase_hists_len(RESULTS_LASE, "8gram")
hist_len_9gram_lase = get_lase_hists_len(RESULTS_LASE, "9gram")
hist_len_10gram_lase = get_lase_hists_len(RESULTS_LASE, "10gram")

325
1058
1607
1933
2126
2236
2299
2332
2348
2361


In [89]:
concat_hists_to_2gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
            
    concat_hists_to_2gram_lase.append(concat_hist)

concat_hists_len_to_2gram_lase = hist_len_1gram_lase + hist_len_2gram_lase

In [90]:
concat_hists_to_3gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
        
    for gram_ind, amount in hists_lase_3gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase, amount))
            
    concat_hists_to_3gram_lase.append(concat_hist)

concat_hists_len_to_3gram_lase = hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase

In [91]:
concat_hists_to_4gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
        
    for gram_ind, amount in hists_lase_3gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase, amount))
        
    for gram_ind, amount in hists_lase_4gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase, amount))
            
    concat_hists_to_4gram_lase.append(concat_hist)

concat_hists_len_to_4gram_lase = hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase + hist_len_4gram_lase

In [92]:
concat_hists_to_5gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
        
    for gram_ind, amount in hists_lase_3gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase, amount))
        
    for gram_ind, amount in hists_lase_4gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase, amount))
        
    for gram_ind, amount in hists_lase_5gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + 
                            hist_len_3gram_lase + hist_len_4gram_lase, 
                            amount))
            
    concat_hists_to_5gram_lase.append(concat_hist)

concat_hists_len_to_5gram_lase = hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase 
concat_hists_len_to_5gram_lase += hist_len_4gram_lase + hist_len_5gram_lase


In [93]:
concat_hists_to_6gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
        
    for gram_ind, amount in hists_lase_3gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase, amount))
        
    for gram_ind, amount in hists_lase_4gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase, amount))
        
    for gram_ind, amount in hists_lase_5gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase + hist_len_4gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_6gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + hist_len_4gram_lase + hist_len_5gram_lase, 
                            amount))
            
    concat_hists_to_6gram_lase.append(concat_hist)

concat_hists_len_to_6gram_lase = hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase
concat_hists_len_to_6gram_lase += hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase


In [94]:
concat_hists_to_7gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
        
    for gram_ind, amount in hists_lase_3gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase, amount))
        
    for gram_ind, amount in hists_lase_4gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase, amount))
        
    for gram_ind, amount in hists_lase_5gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + 
                            hist_len_3gram_lase + hist_len_4gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_6gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + hist_len_4gram_lase + hist_len_5gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_7gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase, 
                            amount))
            
    concat_hists_to_7gram_lase.append(concat_hist)

concat_hists_len_to_7gram_lase = hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase 
concat_hists_len_to_7gram_lase += hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase
concat_hists_len_to_7gram_lase += hist_len_7gram_lase


In [95]:
concat_hists_to_8gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
        
    for gram_ind, amount in hists_lase_3gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase, amount))
        
    for gram_ind, amount in hists_lase_4gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase, amount))
        
    for gram_ind, amount in hists_lase_5gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + 
                            hist_len_3gram_lase + hist_len_4gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_6gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + hist_len_4gram_lase + hist_len_5gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_7gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_8gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase + hist_len_7gram_lase, 
                            amount))
            
    concat_hists_to_8gram_lase.append(concat_hist)

concat_hists_len_to_8gram_lase = hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase 
concat_hists_len_to_8gram_lase += hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase
concat_hists_len_to_8gram_lase += hist_len_7gram_lase + hist_len_8gram_lase


In [96]:
concat_hists_to_9gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
        
    for gram_ind, amount in hists_lase_3gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase, amount))
        
    for gram_ind, amount in hists_lase_4gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase, amount))
        
    for gram_ind, amount in hists_lase_5gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + 
                            hist_len_3gram_lase + hist_len_4gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_6gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + hist_len_4gram_lase + hist_len_5gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_7gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_8gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase + hist_len_7gram_lase, 
                            amount))
    
    for gram_ind, amount in hists_lase_9gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase + 
                            hist_len_7gram_lase + hist_len_8gram_lase, 
                            amount))
            
    concat_hists_to_9gram_lase.append(concat_hist)

concat_hists_len_to_9gram_lase = hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase 
concat_hists_len_to_9gram_lase += hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase
concat_hists_len_to_9gram_lase += hist_len_7gram_lase + hist_len_8gram_lase + hist_len_9gram_lase


In [97]:
concat_hists_to_10gram_lase = []

num_changes = len(hists_lase_1gram)

for i in range(num_changes):
    concat_hist = []
    
    for gram_ind, amount in hists_lase_1gram[i]:
        concat_hist.append((gram_ind, amount))
    
    for gram_ind, amount in hists_lase_2gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase, amount))
        
    for gram_ind, amount in hists_lase_3gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase, amount))
        
    for gram_ind, amount in hists_lase_4gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase, amount))
        
    for gram_ind, amount in hists_lase_5gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + hist_len_2gram_lase + 
                            hist_len_3gram_lase + hist_len_4gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_6gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + hist_len_4gram_lase + hist_len_5gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_7gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_8gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase + hist_len_7gram_lase, 
                            amount))
    
    for gram_ind, amount in hists_lase_9gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase + 
                            hist_len_7gram_lase + hist_len_8gram_lase, 
                            amount))
        
    for gram_ind, amount in hists_lase_10gram[i]:
        concat_hist.append((gram_ind + hist_len_1gram_lase + 
                            hist_len_2gram_lase + hist_len_3gram_lase + 
                            hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase + 
                            hist_len_7gram_lase + hist_len_8gram_lase + hist_len_9gram_lase, 
                            amount))
            
    concat_hists_to_10gram_lase.append(concat_hist)

concat_hists_len_to_10gram_lase = hist_len_1gram_lase + hist_len_2gram_lase + hist_len_3gram_lase 
concat_hists_len_to_10gram_lase += hist_len_4gram_lase + hist_len_5gram_lase + hist_len_6gram_lase
concat_hists_len_to_10gram_lase += hist_len_7gram_lase + hist_len_8gram_lase + hist_len_9gram_lase + hist_len_10gram_lase


In [98]:
concat_hists_lase = []

concat_hists_lase.append([]) # zero array
concat_hists_lase.append(hists_lase_1gram)
concat_hists_lase.append(concat_hists_to_2gram_lase)
concat_hists_lase.append(concat_hists_to_3gram_lase)
concat_hists_lase.append(concat_hists_to_4gram_lase)
concat_hists_lase.append(concat_hists_to_5gram_lase)
concat_hists_lase.append(concat_hists_to_6gram_lase)
concat_hists_lase.append(concat_hists_to_7gram_lase)
concat_hists_lase.append(concat_hists_to_8gram_lase)
concat_hists_lase.append(concat_hists_to_9gram_lase)
concat_hists_lase.append(concat_hists_to_10gram_lase)


In [99]:
concat_hists_lase_len = []

concat_hists_lase_len.append([]) # zero array
concat_hists_lase_len.append(hist_len_1gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_2gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_3gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_4gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_5gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_6gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_7gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_8gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_9gram_lase)
concat_hists_lase_len.append(concat_hists_len_to_10gram_lase)


In [201]:
distances = [jaccard_metric, canberra_metric, cos_distance, pearsons_correlation_mean]
distances_names = ["jaccard_metric", "canberra_metric", "cos_distance", "pearsons_correlation_mean"]

# DBSCAN

In [51]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=False)

        print(best_eps)

        tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                                   min_samples=2, 
                                                   metric='precomputed').fit(cur_dists)
        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


DBSCAN



Best eps: 0.07758620689655171
Best F-measure: 0.19721821123429376
0.07758620689655171
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 5
Number of outliers: 0/149 (0.0%)

Cohesion = 38.8698625099818
Separation = 2737.827542290662
G1 = 277.9643636188017

Entropy = 3.860773
Purity = 0.187919
F-measure = 0.197218

Rand = 20.198%
Jaccard Index = 5.792%



DBSCAN



Best eps: 0.6005464480874317
Best F-measure: 0.9328815852305783
0.6005464480874317
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 4/149 (2.685%)

Cohesion = 49.528446538230305
Separation = 1769.7129054837073
G1 = 236.58235786365387

Entropy = 0.0
Purity = 1.0
F-measure = 0.932882

Rand = 99.282%
Jaccard Index = 85.902%



DBSCAN


...


Best eps: 0.9368169398907104
Best F-measure: 0.9551374847759344
0.9368169398907104
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 27
Number of outliers: 3/149 (2.013%)

Cohesion = 42.36340246565068
Separation = 231.6332537404911
G1 = 31.472214607267585

Entropy = 0.036963
Purity = 0.993151
F-measure = 0.955137

Rand = 99.358%
Jaccard Index = 87.681%



DBSCAN



Best eps: 0.6261725205296185
Best F-measure: 0.8995545096216235
0.6261725205296185
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 51.22623928855977
Separation = 1108.9900737668695
G1 = 167.2538433857298

Entropy = 0.0
Purity = 1.0
F-measure = 0.899555

Rand = 99.251%
Jaccard Index = 85.356%



DBSCAN



Best eps: 0.6752802803214588
Best F-measure: 0.9176753149907511
0.6752802803214588
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 57.44166126228457
Separation = 4142.324994781629
G1 = 482.42906127222955

Entropy = 0.0
Purity = 1.0
F-measure = 0.917675

Rand = 99.34%
Jaccard Index = 87.091%


# Average

In [24]:
cur_dists = get_dists(hists_lase_1gram, jaccard_metric, hist_len_1gram_lase)
link = 'average'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link, with_step=0.005)

print(best_eps)

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)


Best eps: 0.045
Best F-measure: 0.5312122913532581
0.045
CLUSTERS:

1 [1]
2 [1]
3 [1]
4 [1]
31 [5]
32 [5]
34 [5]
35 [5]
42 [5]
64 [11]
65 [11]
66 [11]
114 [20]


5 [2]
7 [2]
8 [2]
13 [2]
19 [2]
20 [2]
94 [16]
96 [16]


6 [2]
12 [2]
14 [2]
18 [2]
21 [3]
22 [3]
23 [3]
24 [3]


9 [2]
15 [2]
43 [6]
44 [6]
45 [6]
61 [10]


10 [2]
11 [2]
16 [2]
17 [2]
33 [5]
36 [5]
37 [5]
40 [5]
116 [20]


25 [4]
27 [4]
30 [4]


26 [4]
28 [4]
29 [4]
113 [20]
115 [20]
122 [21]
144 [24]
145 [24]
146 [24]
148 [24]
149 [24]


38 [5]
82 [14]
83 [14]
84 [14]
88 [16]


39 [5]
41 [5]
92 [16]
93 [16]


46 [7]
47 [7]
48 [7]
49 [7]
50 [7]
51 [7]
52 [7]
53 [8]
54 [8]
55 [8]
56 [8]


58 [9]
59 [9]


62 [10]
63 [10]
67 [12]
68 [12]
69 [12]
73 [12]
74 [12]
75 [12]
95 [16]
112 [20]
117 [21]
118 [21]
119 [21]
120 [21]
121 [21]
123 [21]
124 [21]
125 [21]
126 [21]


70 [12]
71 [12]
72 [12]
77 [13]
78 [13]
79 [13]
80 [13]
81 [13]


76 [13]
89 [16]
91 [16]
97 [17]
98 [17]
99 [17]
100 [17]
130 [23]
131 [23]
132 [23]
133 [23]
134

(92.519, 0.5312122913532581)

In [16]:
cur_dists = get_dists(hists_lase_1gram, canberra_metric, hist_len_1gram_lase)
link = 'average'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link, with_step=0.005)

print(best_eps)

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)


Best eps: 0.685
Best F-measure: 0.9218450453349781
0.685
CLUSTERS:

1 [1]
2 [1]
3 [1]
4 [1]


5 [2]
6 [2]
8 [2]
9 [2]
10 [2]
11 [2]
12 [2]
13 [2]
14 [2]
15 [2]
16 [2]
17 [2]
18 [2]
20 [2]


7 [2]
19 [2]


21 [3]
22 [3]
23 [3]
24 [3]


25 [4]
27 [4]
30 [4]


26 [4]
28 [4]
29 [4]


31 [5]
32 [5]
33 [5]
34 [5]
35 [5]
36 [5]
37 [5]
38 [5]
39 [5]
40 [5]
41 [5]
42 [5]


43 [6]
44 [6]
45 [6]


46 [7]
47 [7]
48 [7]
49 [7]
50 [7]
51 [7]
52 [7]


53 [8]
54 [8]
56 [8]


55 [8]
127 [22]
128 [22]
129 [22]


57 [9]
58 [9]
59 [9]
60 [9]


61 [10]
62 [10]
63 [10]


64 [11]
65 [11]
66 [11]


67 [12]
68 [12]
69 [12]
70 [12]
71 [12]
72 [12]
73 [12]
74 [12]
75 [12]


77 [13]
78 [13]
79 [13]
80 [13]
81 [13]


82 [14]
83 [14]
84 [14]


85 [15]
86 [15]
87 [15]


88 [16]
93 [16]


89 [16]
91 [16]


90 [16]
92 [16]
94 [16]
96 [16]


97 [17]
98 [17]
99 [17]
100 [17]


101 [18]
102 [18]
103 [18]
104 [18]
105 [18]


106 [19]
107 [19]
108 [19]
109 [19]
110 [19]
111 [19]


112 [20]
113 [20]
115 [20]
116 [20]
130 [

(99.216, 0.9218450453349781)

In [52]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'average'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link, with_step=0.005)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.045
Best F-measure: 0.5312122913532581
0.045
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 18
Number of outliers: 4/149 (2.685%)

Cohesion = 63.0310581932956
Separation = 13956.058710356667
G1 = 665.1170988068042

Entropy = 1.337041
Purity = 0.537931
F-measure = 0.531212

Rand = 92.519%
Jaccard Index = 23.656%



Agglomerative Clustering



Best eps: 0.685
Best F-measure: 0.9218450453349781
0.685
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 27
Number of outliers: 3/149 (2.013%)

Cohesion = 48.283696940439334
Separation = 1814.184271443892
G1 = 215.20768927083802

Entropy = 0.083321
Purity = 0.965753
F-measure = 0.921845

Rand = 99.216%
Jaccard Index = 85.125%



Agglomerative Clustering


...


Best eps: 0.935
Best F-measure: 0.9360874073365952
0.935
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 28
Number of outliers: 4/149 (2.685%)

Cohesion = 43.164971237682
Separation = 220.9661676350944
G1 = 31.987432704472404

Entropy = 0.037218
Purity = 0.993103
F-measure = 0.936087

Rand = 99.215%
Jaccard Index = 84.954%



Agglomerative Clustering



Best eps: 0.63
Best F-measure: 0.8995545096216235
0.63
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 51.22623928855977
Separation = 1108.9900737668695
G1 = 167.2538433857298

Entropy = 0.0
Purity = 1.0
F-measure = 0.899555

Rand = 99.251%
Jaccard Index = 85.356%



Agglomerative Clustering



Best eps: 0.67
Best F-measure: 0.8995545096216235
0.67
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 57.90618750503389
Separation = 4126.827846571128
G1 = 509.3592726478762

Entropy = 0.0
Purity = 1.0
F-measure = 0.899555

Rand = 99.251%
Jaccard Index = 85.356%


In [54]:
LASE_ACTIONS_PATH = "/Users/aliscafo/Documents/ALINA/WORK/SPbAU/thesis/CodeDiffEditScripts/LaseActions"

edit_scripts_lase = []

for double_num in tqdm_notebook(ids_per_label):
    actions_file = open(LASE_ACTIONS_PATH + "/" + double_num + "/" + double_num + "/" + "sampleChange1")
    edit_script = actions_file.read().split("\n")
    edit_script = [elem for elem in edit_script if elem != '']
    
    print(len(edit_script), from_change_to_label[double_num])
    
    edit_scripts_lase.append(edit_script)
    actions_file.close()
    

54 [1]
54 [1]
101 [1]
54 [1]
198 [2]
206 [2]
4 [2]
165 [2]
200 [2]
185 [2]
185 [2]
220 [2]
198 [2]
206 [2]
200 [2]
185 [2]
185 [2]
220 [2]
4 [2]
165 [2]
15 [3]
15 [3]
15 [3]
15 [3]
87 [4]
22 [4]
85 [4]
22 [4]
22 [4]
85 [4]
4 [5]
4 [5]
4 [5]
4 [5]
4 [5]
4 [5]
4 [5]
4 [5]
4 [5]
4 [5]
4 [5]
4 [5]
43 [6]
43 [6]
43 [6]
212 [7]
211 [7]
211 [7]
211 [7]
211 [7]
211 [7]
212 [7]
3 [8]
3 [8]
28 [8]
3 [8]
180 [9]
134 [9]
134 [9]
164 [9]
12 [10]
12 [10]
12 [10]
40 [11]
40 [11]
51 [11]
71 [12]
39 [12]
39 [12]
72 [12]
56 [12]
56 [12]
22 [12]
22 [12]
22 [12]
7 [13]
22 [13]
22 [13]
22 [13]
22 [13]
22 [13]
113 [14]
113 [14]
137 [14]
140 [15]
140 [15]
246 [15]
21 [16]
14 [16]
81 [16]
14 [16]
80 [16]
41 [16]
44 [16]
11 [16]
44 [16]
10 [17]
10 [17]
10 [17]
10 [17]
49 [18]
49 [18]
54 [18]
49 [18]
49 [18]
11 [19]
11 [19]
11 [19]
11 [19]
11 [19]
11 [19]
12 [20]
12 [20]
4 [20]
12 [20]
12 [20]
28 [21]
23 [21]
28 [21]
28 [21]
28 [21]
37 [21]
28 [21]
23 [21]
23 [21]
23 [21]
44 [22]
44 [22]
47 [22]
13 [23]
13 [23]

In [19]:
def lcs(X, Y): 
    # find the length of the strings 
    m = len(X) 
    n = len(Y) 
  
    # declaring the array for storing the dp values 
    L = [[None]*(n + 1) for i in range(m + 1)] 
  
    """Following steps build L[m + 1][n + 1] in bottom up fashion 
    Note: L[i][j] contains length of LCS of X[0..i-1] 
    and Y[0..j-1]"""
    for i in range(m + 1): 
        for j in range(n + 1): 
            if i == 0 or j == 0 : 
                L[i][j] = 0
            elif X[i-1] == Y[j-1]: 
                L[i][j] = L[i-1][j-1]+1
            else: 
                L[i][j] = max(L[i-1][j], L[i][j-1]) 
  
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1] 
    return L[m][n] 

In [20]:
dists_for_lcs_lase = []

for es1 in tqdm_notebook(edit_scripts_lase):
    cur_dists = []
    for es2 in edit_scripts_lase:
        cur_lcs = lcs(es1, es2)
        dist = 1 - cur_lcs / max(len(es1), len(es2))
        cur_dists.append(dist)
    dists_for_lcs_lase.append(cur_dists)

In [21]:
cur_dists = dists_for_lcs_lase
link = 'average'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link, with_step=0.005)

print(best_eps)

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)


Best eps: 0.525
Best F-measure: 0.9147607798614508
0.525
CLUSTERS:

1 [1]
2 [1]
3 [1]
4 [1]


5 [2]
6 [2]
8 [2]
9 [2]
10 [2]
11 [2]
12 [2]
13 [2]
14 [2]
15 [2]
16 [2]
17 [2]
18 [2]
20 [2]


7 [2]
19 [2]


21 [3]
22 [3]
23 [3]
24 [3]


25 [4]
27 [4]
30 [4]


26 [4]
28 [4]
29 [4]


31 [5]
32 [5]
33 [5]
34 [5]
35 [5]
36 [5]
37 [5]
38 [5]
39 [5]
40 [5]
41 [5]
42 [5]


43 [6]
44 [6]
45 [6]


46 [7]
47 [7]
48 [7]
49 [7]
50 [7]
51 [7]
52 [7]


53 [8]
54 [8]
56 [8]


57 [9]
58 [9]
59 [9]
60 [9]


61 [10]
62 [10]
63 [10]


64 [11]
65 [11]
66 [11]


67 [12]
70 [12]
71 [12]
72 [12]


68 [12]
69 [12]


73 [12]
74 [12]
75 [12]


77 [13]
78 [13]
79 [13]
80 [13]
81 [13]


82 [14]
83 [14]
84 [14]


85 [15]
86 [15]
87 [15]


88 [16]
89 [16]
91 [16]


90 [16]
92 [16]


94 [16]
96 [16]


97 [17]
98 [17]
99 [17]
100 [17]


101 [18]
102 [18]
103 [18]
104 [18]
105 [18]


106 [19]
107 [19]
108 [19]
109 [19]
110 [19]
111 [19]


112 [20]
113 [20]
115 [20]
116 [20]


117 [21]
118 [21]
119 [21]
120 [21]
121 [21

(99.233, 0.9147607798614508)

# Complete

In [22]:
cur_dists = get_dists(hists_lase_1gram, canberra_metric, hist_len_1gram_lase)
link = 'complete'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link)

print(best_eps)

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)


Best eps: 0.6538517398907009
Best F-measure: 0.9328815852305783
0.6538517398907009
CLUSTERS:

1 [1]
2 [1]
3 [1]
4 [1]


5 [2]
6 [2]
8 [2]
9 [2]
10 [2]
11 [2]
12 [2]
13 [2]
14 [2]
15 [2]
16 [2]
17 [2]
18 [2]
20 [2]


7 [2]
19 [2]


21 [3]
22 [3]
23 [3]
24 [3]


25 [4]
27 [4]
30 [4]


26 [4]
28 [4]
29 [4]


31 [5]
32 [5]
33 [5]
34 [5]
35 [5]
36 [5]
37 [5]
38 [5]
39 [5]
40 [5]
41 [5]
42 [5]


43 [6]
44 [6]
45 [6]


46 [7]
47 [7]
48 [7]
49 [7]
50 [7]
51 [7]
52 [7]


53 [8]
54 [8]
56 [8]


57 [9]
58 [9]
59 [9]
60 [9]


61 [10]
62 [10]
63 [10]


64 [11]
65 [11]
66 [11]


67 [12]
68 [12]
69 [12]
70 [12]
71 [12]
72 [12]


73 [12]
74 [12]
75 [12]


77 [13]
78 [13]
79 [13]
80 [13]
81 [13]


82 [14]
83 [14]
84 [14]


85 [15]
86 [15]
87 [15]


88 [16]
93 [16]


89 [16]
91 [16]


90 [16]
92 [16]
94 [16]
96 [16]


97 [17]
98 [17]
99 [17]
100 [17]


101 [18]
102 [18]
103 [18]
104 [18]
105 [18]


106 [19]
107 [19]
108 [19]
109 [19]
110 [19]
111 [19]


112 [20]
113 [20]
115 [20]
116 [20]


117 [21]
11

(99.282, 0.9328815852305783)

In [23]:
cur_dists = dists_for_lcs_lase
link = 'complete'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link)

print(best_eps)

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)


Best eps: 0.5384615384615384
Best F-measure: 0.9147607798614508
0.5384615384615384
CLUSTERS:

1 [1]
2 [1]
3 [1]
4 [1]


5 [2]
6 [2]
8 [2]
9 [2]
10 [2]
11 [2]
12 [2]
13 [2]
14 [2]
15 [2]
16 [2]
17 [2]
18 [2]
20 [2]


7 [2]
19 [2]


21 [3]
22 [3]
23 [3]
24 [3]


25 [4]
27 [4]
30 [4]


26 [4]
28 [4]
29 [4]


31 [5]
32 [5]
33 [5]
34 [5]
35 [5]
36 [5]
37 [5]
38 [5]
39 [5]
40 [5]
41 [5]
42 [5]


43 [6]
44 [6]
45 [6]


46 [7]
47 [7]
48 [7]
49 [7]
50 [7]
51 [7]
52 [7]


53 [8]
54 [8]
56 [8]


57 [9]
58 [9]
59 [9]
60 [9]


61 [10]
62 [10]
63 [10]


64 [11]
65 [11]
66 [11]


67 [12]
70 [12]
71 [12]
72 [12]


68 [12]
69 [12]


73 [12]
74 [12]
75 [12]


77 [13]
78 [13]
79 [13]
80 [13]
81 [13]


82 [14]
83 [14]
84 [14]


85 [15]
86 [15]
87 [15]


88 [16]
89 [16]
91 [16]


90 [16]
92 [16]


94 [16]
96 [16]


97 [17]
98 [17]
99 [17]
100 [17]


101 [18]
102 [18]
103 [18]
104 [18]
105 [18]


106 [19]
107 [19]
108 [19]
109 [19]
110 [19]
111 [19]


112 [20]
113 [20]
115 [20]
116 [20]


117 [21]
118 [21]

(99.233, 0.9147607798614508)

In [53]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'complete'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.29166666666666663
Best F-measure: 0.5425752440596499
0.29166666666666663
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 19
Number of outliers: 0/149 (0.0%)

Cohesion = 64.07092200549252
Separation = 14415.480193668323
G1 = 784.3392634900117

Entropy = 1.419877
Purity = 0.557047
F-measure = 0.542575

Rand = 91.983%
Jaccard Index = 22.997%



Agglomerative Clustering



Best eps: 0.6538517398907009
Best F-measure: 0.9328815852305783
0.6538517398907009
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 4/149 (2.685%)

Cohesion = 49.528446538230305
Separation = 1769.7129054837073
G1 = 236.58235786365387

Entropy = 0.0
Purity = 1.0
F-measure = 0.932882

Rand = 99.282%
Jaccard Index = 85.902%



Agglomerative Clustering


...


Best eps: 0.9343283582089552
Best F-measure: 0.9212140581268096
0.9343283582089552
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 5/149 (3.356%)

Cohesion = 43.97713374933263
Separation = 217.79345922932353
G1 = 33.779128282288454

Entropy = 0.0
Purity = 1.0
F-measure = 0.921214

Rand = 99.31%
Jaccard Index = 86.476%



Agglomerative Clustering



Best eps: 0.6293820663876157
Best F-measure: 0.8995545096216235
0.6293820663876157
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 51.22623928855977
Separation = 1108.9900737668695
G1 = 167.2538433857298

Entropy = 0.0
Purity = 1.0
F-measure = 0.899555

Rand = 99.251%
Jaccard Index = 85.356%



Agglomerative Clustering



Best eps: 0.8267994728456971
Best F-measure: 0.9008880753847194
0.8267994728456971
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 27
Number of outliers: 4/149 (2.685%)

Cohesion = 56.57278691952854
Separation = 4359.12341034228
G1 = 429.0075755366955

Entropy = 0.082759
Purity = 0.958621
F-measure = 0.900888

Rand = 99.1%
Jaccard Index = 82.909%


# LASE with UPD

### DBSCAN

In [81]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=False)

        print(best_eps)

        tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                                   min_samples=2, 
                                                   metric='precomputed').fit(cur_dists)
        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


DBSCAN



Best eps: 0.06149425287356314
Best F-measure: 0.18754906921092465
0.06149425287356314
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 5
Number of outliers: 1/149 (0.671%)

Cohesion = 37.89767092344934
Separation = 2367.1412008881625
G1 = 362.0127455303709

Entropy = 3.944602
Purity = 0.175676
F-measure = 0.187549

Rand = 17.834%
Jaccard Index = 5.658%



DBSCAN



Best eps: 0.6005464480874317
Best F-measure: 0.9328815852305783
0.6005464480874317
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 4/149 (2.685%)

Cohesion = 49.49948347777997
Separation = 1730.2662643789113
G1 = 230.7760864177212

Entropy = 0.0
Purity = 1.0
F-measure = 0.932882

Rand = 99.282%
Jaccard Index = 85.902%



DBSCAN


...


Best eps: 0.9368169398907104
Best F-measure: 0.9551374847759344
0.9368169398907104
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 27
Number of outliers: 3/149 (2.013%)

Cohesion = 42.37029467536296
Separation = 228.97948296516387
G1 = 30.97477199244008

Entropy = 0.036963
Purity = 0.993151
F-measure = 0.955137

Rand = 99.358%
Jaccard Index = 87.681%



DBSCAN



Best eps: 0.5096685771120242
Best F-measure: 0.8957194185382102
0.5096685771120242
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 7/149 (4.698%)

Cohesion = 51.13059732945803
Separation = 1018.2671954142439
G1 = 152.8816512443778

Entropy = 0.0
Purity = 1.0
F-measure = 0.895719

Rand = 99.241%
Jaccard Index = 85.271%



DBSCAN



Best eps: 0.6924951006225576
Best F-measure: 0.9042524961988047
0.6924951006225576
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 28
Number of outliers: 6/149 (4.027%)

Cohesion = 56.96370406987147
Separation = 4074.3500116170258
G1 = 439.41404740743087

Entropy = 0.041958
Purity = 0.979021
F-measure = 0.904252

Rand = 99.251%
Jaccard Index = 85.606%


### HAC average

In [84]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'average'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link, with_step=0.005)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.03
Best F-measure: 0.5703295859730917
0.03
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 23
Number of outliers: 5/149 (3.356%)

Cohesion = 60.31539412669273
Separation = 14276.9871916422
G1 = 1477.6441713998092

Entropy = 1.18215
Purity = 0.590278
F-measure = 0.57033

Rand = 93.056%
Jaccard Index = 26.213%



Agglomerative Clustering



Best eps: 0.705
Best F-measure: 0.9352678641269244
0.705
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 26
Number of outliers: 3/149 (2.013%)

Cohesion = 47.73324417810008
Separation = 1773.5763188071564
G1 = 199.68717794822118

Entropy = 0.083321
Purity = 0.965753
F-measure = 0.935268

Rand = 99.301%
Jaccard Index = 86.738%



Agglomerative Clustering


...


Best eps: 0.935
Best F-measure: 0.9360874073365952
0.935
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 28
Number of outliers: 4/149 (2.685%)

Cohesion = 43.1728549425342
Separation = 218.52842007798483
G1 = 31.509227849269895

Entropy = 0.037218
Purity = 0.993103
F-measure = 0.936087

Rand = 99.215%
Jaccard Index = 84.954%



Agglomerative Clustering



Best eps: 0.63
Best F-measure: 0.8995545096216235
0.63
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 51.16023109982057
Separation = 1044.6961859193798
G1 = 154.23358723521727

Entropy = 0.0
Purity = 1.0
F-measure = 0.899555

Rand = 99.251%
Jaccard Index = 85.356%



Agglomerative Clustering



Best eps: 0.8250000000000001
Best F-measure: 0.9098046621536552
0.8250000000000001
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 28
Number of outliers: 3/149 (2.013%)

Cohesion = 57.03581140720807
Separation = 4354.353043094204
G1 = 447.7274630596303

Entropy = 0.069464
Purity = 0.972603
F-measure = 0.909805

Rand = 99.112%
Jaccard Index = 82.909%


### HAC complete

In [85]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'complete'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.25396825396825395
Best F-measure: 0.5510879027963193
0.25396825396825395
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 19
Number of outliers: 0/149 (0.0%)

Cohesion = 64.26128686741748
Separation = 14489.14902811083
G1 = 711.9665533803262

Entropy = 1.321847
Purity = 0.543624
F-measure = 0.551088

Rand = 92.708%
Jaccard Index = 25.07%



Agglomerative Clustering



Best eps: 0.6553987678987678
Best F-measure: 0.9328815852305783
0.6553987678987678
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 4/149 (2.685%)

Cohesion = 49.49948347777997
Separation = 1730.2662643789113
G1 = 230.7760864177212

Entropy = 0.0
Purity = 1.0
F-measure = 0.932882

Rand = 99.282%
Jaccard Index = 85.902%



Agglomerative Clustering


...


Best eps: 0.9351578947368421
Best F-measure: 0.9212140581268096
0.9351578947368421
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 5/149 (3.356%)

Cohesion = 43.98871766838173
Separation = 215.37588351638337
G1 = 33.23429039851587

Entropy = 0.0
Purity = 1.0
F-measure = 0.921214

Rand = 99.31%
Jaccard Index = 86.476%



Agglomerative Clustering



Best eps: 0.6371145751670915
Best F-measure: 0.8995545096216235
0.6371145751670915
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 51.16023109982057
Separation = 1044.6961859193798
G1 = 154.23358723521727

Entropy = 0.0
Purity = 1.0
F-measure = 0.899555

Rand = 99.251%
Jaccard Index = 85.356%



Agglomerative Clustering



Best eps: 0.6759849436351233
Best F-measure: 0.8995545096216235
0.6759849436351233
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 57.80991851779305
Separation = 4047.979664766859
G1 = 496.01846631750806

Entropy = 0.0
Purity = 1.0
F-measure = 0.899555

Rand = 99.251%
Jaccard Index = 85.356%


In [183]:
LASE_ACTIONS_PATH = "/Users/aliscafo/Documents/ALINA/WORK/SPbAU/thesis/CodeDiffEditScripts/LaseActionsWithUPD"

edit_scripts_lase = []

for double_num in tqdm_notebook(ids_per_label):
    actions_file = open(LASE_ACTIONS_PATH + "/" + double_num + "/" + double_num + "/" + "sampleChange1")
    edit_script = actions_file.read().split("\n")
    edit_script = [elem for elem in edit_script if elem != '']
    
    print(len(edit_script), from_change_to_label[double_num])
    print(edit_script)
    
    edit_scripts_lase.append(edit_script)
    actions_file.close()
    

54 [1]
['UPD 27@@ 25@@', 'INS 34@@ 27@@ at 1', 'INS 60@@ 8@@ at 0', 'INS 24@@ 8@@ at 1', 'UPD 42@@ 22@@', 'MOV 5@@ 60@@ at 0', 'INS 59@@ 60@@ at 1', 'INS 58@@ 24@@ at 0', 'INS 27@@ 24@@ at 1', 'INS 37@@ 24@@ at 2', 'INS 8@@ 24@@ at 3', 'MOV 42@@ 59@@ at 0', 'INS 33@@ 59@@ at 1', 'INS 39@@ 58@@ at 0', 'INS 59@@ 58@@ at 1', 'INS 27@@ 27@@ at 0', 'INS 27@@ 27@@ at 1', 'INS 42@@ 37@@ at 0', 'INS 25@@ 8@@ at 0', 'INS 42@@ 59@@ at 0', 'INS 34@@ 59@@ at 1', 'INS 42@@ 27@@ at 0', 'INS 33@@ 27@@ at 1', 'INS 42@@ 27@@ at 0', 'INS 22@@ 27@@ at 1', 'INS 27@@ 25@@ at 0', 'INS 21@@ 25@@ at 1', 'INS 52@@ 22@@ at 0', 'INS 42@@ 22@@ at 1', 'INS 2@@ 27@@ at 0', 'INS 42@@ 27@@ at 1', 'INS 7@@ 21@@ at 0', 'INS 22@@ 2@@ at 0', 'INS 42@@ 2@@ at 1', 'INS 42@@ 7@@ at 0', 'INS 2@@ 7@@ at 1', 'MOV 52@@ 22@@ at 0', 'UPD 42@@ 22@@', 'MOV 42@@ 22@@ at 1', 'INS 22@@ 2@@ at 0', 'INS 42@@ 2@@ at 1', 'INS 52@@ 22@@ at 0', 'INS 42@@ 22@@ at 1', 'DEL 33@@ 27@@', 'DEL 39@@ 5@@', 'DEL 85@@ 5@@', 'DEL 5@@ 11@@', 'DEL 22@@ 

In [184]:
dists_for_lcs_lase = []

for es1 in tqdm_notebook(edit_scripts_lase):
    cur_dists = []
    for es2 in edit_scripts_lase:
        cur_lcs = lcs(es1, es2)
        dist = 1 - cur_lcs / max(len(es1), len(es2))
        cur_dists.append(dist)
    dists_for_lcs_lase.append(cur_dists)

In [185]:
cur_dists = dists_for_lcs_lase
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                            from_change_to_label, ids_per_label, 
                            agglomerative=False)

print(best_eps)

tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                           min_samples=2, 
                                           metric='precomputed').fit(cur_dists)
print("_______________________________")
print(i, dist_name)
print("_______________________________")
print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)

DBSCAN



Best eps: 0.5238095238095238
Best F-measure: 0.9259120447039908
0.5238095238095238
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 29
Number of outliers: 5/149 (3.356%)

Cohesion = 50.998737399493564
Separation = 1769.862618338729
G1 = 215.56693366636847

Entropy = 0.0
Purity = 1.0
F-measure = 0.925912

Rand = 99.31%
Jaccard Index = 86.476%


(99.31, 0.9259120447039908)

In [186]:
cur_dists = dists_for_lcs_lase
link = 'average'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                            from_change_to_label, ids_per_label, 
                            agglomerative=True, linkage=link, with_step=0.005)

print(best_eps)

print("_______________________________")
print(i, dist_name)
print("_______________________________")

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                    affinity='precomputed', 
                                                    linkage=link, 
                                                    compute_full_tree=True,
                                                    distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)


Agglomerative Clustering



Best eps: 0.525
Best F-measure: 0.9147607798614508
0.525
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 5/149 (3.356%)

Cohesion = 51.14172925375302
Separation = 1778.4948417276805
G1 = 233.51329149912507

Entropy = 0.0
Purity = 1.0
F-measure = 0.914761

Rand = 99.233%
Jaccard Index = 84.952%


(99.233, 0.9147607798614508)

In [187]:
cur_dists = dists_for_lcs_lase
link = 'complete'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                            from_change_to_label, ids_per_label, 
                            agglomerative=True, linkage=link)

print(best_eps)

print("_______________________________")
print(i, dist_name)
print("_______________________________")

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                    affinity='precomputed', 
                                                    linkage=link, 
                                                    compute_full_tree=True,
                                                    distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)


Agglomerative Clustering



Best eps: 0.5384615384615384
Best F-measure: 0.9147607798614508
0.5384615384615384
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 5/149 (3.356%)

Cohesion = 51.14172925375302
Separation = 1778.4948417276805
G1 = 233.51329149912507

Entropy = 0.0
Purity = 1.0
F-measure = 0.914761

Rand = 99.233%
Jaccard Index = 84.952%


(99.233, 0.9147607798614508)

In [118]:
dists_for_equals_lase = []

for es1 in tqdm_notebook(edit_scripts_lase):
    cur_dists = []
    for es2 in edit_scripts_lase:
        if es1 == es2:
            dist = 0.0
        else:
            dist = 1.0
        cur_dists.append(dist)
    dists_for_equals_lase.append(cur_dists)

In [124]:
cur_dists = dists_for_equals_lase

best_eps = 0.1

tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                           min_samples=2, 
                                           metric='precomputed').fit(cur_dists)
print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)

Number of clusters: 33
Number of outliers: 33/149 (22.148%)

Cohesion = 41.5
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.696348

Rand = 97.166%
Jaccard Index = 54.126%


(97.166, 0.696348059099737)

In [122]:
cur_dists = dists_for_equals_lase
link = 'average'

best_eps = 0.1
tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                    affinity='precomputed', 
                                                    linkage=link, 
                                                    compute_full_tree=True,
                                                    distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)


Number of clusters: 33
Number of outliers: 33/149 (22.148%)

Cohesion = 41.5
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.696348

Rand = 97.166%
Jaccard Index = 54.126%


(97.166, 0.696348059099737)

In [123]:
cur_dists = dists_for_equals_lase
link = 'complete'

best_eps = 0.1

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                    affinity='precomputed', 
                                                    linkage=link, 
                                                    compute_full_tree=True,
                                                    distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)


Number of clusters: 33
Number of outliers: 33/149 (22.148%)

Cohesion = 41.5
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.696348

Rand = 97.166%
Jaccard Index = 54.126%


(97.166, 0.696348059099737)

# LASE without context

### DBSCAN

In [102]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=False)

        print(best_eps)

        tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                                   min_samples=2, 
                                                   metric='precomputed').fit(cur_dists)
        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


DBSCAN



Best eps: 0.08593073593073586
Best F-measure: 0.2216340614071988
0.08593073593073586
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 6
Number of outliers: 0/149 (0.0%)

Cohesion = 39.60702510537168
Separation = 2836.0744018749474
G1 = 397.64631655220364

Entropy = 3.753627
Purity = 0.208054
F-measure = 0.221634

Rand = 23.753%
Jaccard Index = 6.004%



DBSCAN



Best eps: 0.6005464480874317
Best F-measure: 0.94496212214333
0.6005464480874317
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 28
Number of outliers: 4/149 (2.685%)

Cohesion = 48.83444955205416
Separation = 1997.2847510863821
G1 = 250.72515737475507

Entropy = 0.0
Purity = 1.0
F-measure = 0.944962

Rand = 99.454%
Jaccard Index = 89.286%



DBSCAN


...


Best eps: 0.9351851851851852
Best F-measure: 0.9551374847759344
0.9351851851851852
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 27
Number of outliers: 3/149 (2.013%)

Cohesion = 42.327850824548875
Separation = 253.5276520504006
G1 = 33.601532897768024

Entropy = 0.036963
Purity = 0.993151
F-measure = 0.955137

Rand = 99.358%
Jaccard Index = 87.681%



DBSCAN



Best eps: 0.4943876986268312
Best F-measure: 0.8876594937558635
0.4943876986268312
_______________________________
10 cos_distance
_______________________________
Number of clusters: 28
Number of outliers: 7/149 (4.698%)

Cohesion = 50.18563787927453
Separation = 1699.418782424467
G1 = 203.55362719238772

Entropy = 0.082806
Purity = 0.964789
F-measure = 0.887659

Rand = 98.981%
Jaccard Index = 81.488%



DBSCAN



Best eps: 0.5486453034983856
Best F-measure: 0.8876594937558635
0.5486453034983856
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 28
Number of outliers: 7/149 (4.698%)

Cohesion = 54.67664583373232
Separation = 3621.01318040767
G1 = 403.18375524033803

Entropy = 0.082806
Purity = 0.964789
F-measure = 0.887659

Rand = 98.981%
Jaccard Index = 81.488%


### HAC average

In [103]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'average'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link, with_step=0.005)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.035
Best F-measure: 0.5526319451806876
0.035
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 27
Number of outliers: 4/149 (2.685%)

Cohesion = 58.699828929147195
Separation = 14613.14806804667
G1 = 1591.6382427545284

Entropy = 1.13823
Purity = 0.613793
F-measure = 0.552632

Rand = 93.161%
Jaccard Index = 22.644%



Agglomerative Clustering



Best eps: 0.635
Best F-measure: 0.924721363647538
0.635
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 27
Number of outliers: 4/149 (2.685%)

Cohesion = 48.00401363599733
Separation = 1999.1812008376771
G1 = 239.07032011993098

Entropy = 0.061515
Purity = 0.972414
F-measure = 0.924721

Rand = 99.262%
Jaccard Index = 86.051%



Agglomerative Clustering


...


Best eps: 0.93
Best F-measure: 0.9360874073365952
0.93
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 28
Number of outliers: 4/149 (2.685%)

Cohesion = 43.13164387527553
Separation = 241.925139932531
G1 = 34.2006740611084

Entropy = 0.037218
Purity = 0.993103
F-measure = 0.936087

Rand = 99.215%
Jaccard Index = 84.954%



Agglomerative Clustering



Best eps: 0.4
Best F-measure: 0.8867708726769126
0.4
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 9/149 (6.04%)

Cohesion = 51.13475115849401
Separation = 1674.9959690577577
G1 = 250.06929671522315

Entropy = 0.0
Purity = 1.0
F-measure = 0.886771

Rand = 99.26%
Jaccard Index = 85.799%



Agglomerative Clustering



Best eps: 0.42
Best F-measure: 0.8867708726769126
0.42
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 9/149 (6.04%)

Cohesion = 55.59499138801701
Separation = 3534.315316801324
G1 = 460.0667905453221

Entropy = 0.0
Purity = 1.0
F-measure = 0.886771

Rand = 99.26%
Jaccard Index = 85.799%


### HAC complete

In [104]:
for i in range(1, 11):
    hists = concat_hists_lase[i]
    hists_len = concat_hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'complete'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.25555555555555554
Best F-measure: 0.5773679592767631
0.25555555555555554
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 23
Number of outliers: 0/149 (0.0%)

Cohesion = 62.321974252005546
Separation = 15066.229912630779
G1 = 881.8877402018372

Entropy = 1.087539
Purity = 0.610738
F-measure = 0.577368

Rand = 93.978%
Jaccard Index = 25.893%



Agglomerative Clustering



Best eps: 0.6851851851851851
Best F-measure: 0.9300052669180184
0.6851851851851851
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 3/149 (2.013%)

Cohesion = 49.20290456166456
Separation = 2065.7927233867895
G1 = 266.8154263400863

Entropy = 0.022227
Purity = 0.993151
F-measure = 0.930005

Rand = 99.235%
Jaccard Index = 84.944%



Agglomerative Clustering


...


Best eps: 0.9651741293532337
Best F-measure: 0.9247057092434878
0.9651741293532337
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 26
Number of outliers: 2/149 (1.342%)

Cohesion = 40.374851234369956
Separation = 257.6225521271974
G1 = 32.29429581115038

Entropy = 0.13199
Purity = 0.959184
F-measure = 0.924706

Rand = 99.022%
Jaccard Index = 82.143%



Agglomerative Clustering



Best eps: 0.4467854078516117
Best F-measure: 0.8867708726769126
0.4467854078516117
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 9/149 (6.04%)

Cohesion = 51.13475115849401
Separation = 1674.9959690577577
G1 = 250.06929671522315

Entropy = 0.0
Purity = 1.0
F-measure = 0.886771

Rand = 99.26%
Jaccard Index = 85.799%



Agglomerative Clustering



Best eps: 0.47766243452861745
Best F-measure: 0.8867708726769126
0.47766243452861745
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 9/149 (6.04%)

Cohesion = 55.59499138801701
Separation = 3534.315316801324
G1 = 460.0667905453221

Entropy = 0.0
Purity = 1.0
F-measure = 0.886771

Rand = 99.26%
Jaccard Index = 85.799%


In [182]:
LASE_ACTIONS_PATH = "/Users/aliscafo/Documents/ALINA/WORK/SPbAU/thesis/CodeDiffEditScripts/LaseActionsWoContext"

edit_scripts_lase = []

for double_num in tqdm_notebook(ids_per_label):
    actions_file = open(LASE_ACTIONS_PATH + "/" + double_num + "/" + double_num + "/" + "sampleChange1")
    edit_script = actions_file.read().split("\n")
    edit_script = [elem for elem in edit_script if elem != '']
    
    print(len(edit_script), from_change_to_label[double_num])
    print(edit_script)
    
    edit_scripts_lase.append(edit_script)
    actions_file.close()
    

54 [1]
['UPD 27@@', 'INS 34@@ 27@@ at 1', 'INS 60@@ 8@@ at 0', 'INS 24@@ 8@@ at 1', 'UPD 42@@', 'MOV 5@@ 60@@ at 0', 'INS 59@@ 60@@ at 1', 'INS 58@@ 24@@ at 0', 'INS 27@@ 24@@ at 1', 'INS 37@@ 24@@ at 2', 'INS 8@@ 24@@ at 3', 'MOV 42@@ 59@@ at 0', 'INS 33@@ 59@@ at 1', 'INS 39@@ 58@@ at 0', 'INS 59@@ 58@@ at 1', 'INS 27@@ 27@@ at 0', 'INS 27@@ 27@@ at 1', 'INS 42@@ 37@@ at 0', 'INS 25@@ 8@@ at 0', 'INS 42@@ 59@@ at 0', 'INS 34@@ 59@@ at 1', 'INS 42@@ 27@@ at 0', 'INS 33@@ 27@@ at 1', 'INS 42@@ 27@@ at 0', 'INS 22@@ 27@@ at 1', 'INS 27@@ 25@@ at 0', 'INS 21@@ 25@@ at 1', 'INS 52@@ 22@@ at 0', 'INS 42@@ 22@@ at 1', 'INS 2@@ 27@@ at 0', 'INS 42@@ 27@@ at 1', 'INS 7@@ 21@@ at 0', 'INS 22@@ 2@@ at 0', 'INS 42@@ 2@@ at 1', 'INS 42@@ 7@@ at 0', 'INS 2@@ 7@@ at 1', 'MOV 52@@ 22@@ at 0', 'UPD 42@@', 'MOV 42@@ 22@@ at 1', 'INS 22@@ 2@@ at 0', 'INS 42@@ 2@@ at 1', 'INS 52@@ 22@@ at 0', 'INS 42@@ 22@@ at 1', 'DEL 33@@', 'DEL 39@@', 'DEL 85@@', 'DEL 5@@', 'DEL 22@@', 'DEL 42@@', 'DEL 42@@', 'DEL 32

In [170]:
dists_for_lcs_lase = []

for es1 in tqdm_notebook(edit_scripts_lase):
    cur_dists = []
    for es2 in edit_scripts_lase:
        cur_lcs = lcs(es1, es2)
        dist = 1 - cur_lcs / max(len(es1), len(es2))
        cur_dists.append(dist)
    dists_for_lcs_lase.append(cur_dists)

In [172]:
cur_dists = dists_for_lcs_lase
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                            from_change_to_label, ids_per_label, 
                            agglomerative=False)

print(best_eps)

tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                           min_samples=2, 
                                           metric='precomputed').fit(cur_dists)
print("_______________________________")
print(i, dist_name)
print("_______________________________")
print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)

DBSCAN



Best eps: 0.5238095238095238
Best F-measure: 0.9259120447039908
0.5238095238095238
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 29
Number of outliers: 5/149 (3.356%)

Cohesion = 51.096407171642966
Separation = 2103.30578719415
G1 = 249.5272577600066

Entropy = 0.0
Purity = 1.0
F-measure = 0.925912

Rand = 99.31%
Jaccard Index = 86.476%


(99.31, 0.9259120447039908)

In [173]:
cur_dists = dists_for_lcs_lase
link = 'average'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                            from_change_to_label, ids_per_label, 
                            agglomerative=True, linkage=link, with_step=0.005)

print(best_eps)

print("_______________________________")
print(i, dist_name)
print("_______________________________")

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                    affinity='precomputed', 
                                                    linkage=link, 
                                                    compute_full_tree=True,
                                                    distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)


Agglomerative Clustering



Best eps: 0.525
Best F-measure: 0.9147607798614508
0.525
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 5/149 (3.356%)

Cohesion = 51.23827422152058
Separation = 2109.7280393234605
G1 = 275.49397659773774

Entropy = 0.0
Purity = 1.0
F-measure = 0.914761

Rand = 99.233%
Jaccard Index = 84.952%


(99.233, 0.9147607798614508)

In [174]:
cur_dists = dists_for_lcs_lase
link = 'complete'
best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                            from_change_to_label, ids_per_label, 
                            agglomerative=True, linkage=link)

print(best_eps)

print("_______________________________")
print(i, dist_name)
print("_______________________________")

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                    affinity='precomputed', 
                                                    linkage=link, 
                                                    compute_full_tree=True,
                                                    distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)


Agglomerative Clustering



Best eps: 0.5384615384615384
Best F-measure: 0.9147607798614508
0.5384615384615384
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 5/149 (3.356%)

Cohesion = 51.23827422152058
Separation = 2109.7280393234605
G1 = 275.49397659773774

Entropy = 0.0
Purity = 1.0
F-measure = 0.914761

Rand = 99.233%
Jaccard Index = 84.952%


(99.233, 0.9147607798614508)

In [178]:
dists_for_equals_lase = []

for es1 in tqdm_notebook(edit_scripts_lase):
    cur_dists = []
    for es2 in edit_scripts_lase:
        if es1 == es2:
            dist = 0.0
        else:
            dist = 1.0
        cur_dists.append(dist)
    dists_for_equals_lase.append(cur_dists)

In [179]:
cur_dists = dists_for_equals_lase

best_eps = 0.1

tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                           min_samples=2, 
                                           metric='precomputed').fit(cur_dists)
print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)

Number of clusters: 33
Number of outliers: 33/149 (22.148%)

Cohesion = 41.5
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.696348

Rand = 97.166%
Jaccard Index = 54.126%


(97.166, 0.696348059099737)

In [180]:
cur_dists = dists_for_equals_lase
link = 'average'

best_eps = 0.1
tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                    affinity='precomputed', 
                                                    linkage=link, 
                                                    compute_full_tree=True,
                                                    distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)


Number of clusters: 33
Number of outliers: 33/149 (22.148%)

Cohesion = 41.5
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.696348

Rand = 97.166%
Jaccard Index = 54.126%


(97.166, 0.696348059099737)

In [181]:
cur_dists = dists_for_equals_lase
link = 'complete'

best_eps = 0.1

tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                    affinity='precomputed', 
                                                    linkage=link, 
                                                    compute_full_tree=True,
                                                    distance_threshold=best_eps).fit(cur_dists)

print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                        label_to_changes, from_change_to_label, ids_per_label,
                              without_outliers=True)


Number of clusters: 33
Number of outliers: 33/149 (22.148%)

Cohesion = 41.5
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.696348

Rand = 97.166%
Jaccard Index = 54.126%


(97.166, 0.696348059099737)

# LASE for fixed n (with context)

In [202]:
#RESULTS_LASE = "/Volumes/Seagate/Alina/result_for_lase_with_context_nonconcat"

RESULTS_LASE = "/Volumes/Seagate/Alina/result_for_lase_wo_context_nonconcat"

In [203]:
hists_lase_1gram = get_lase_hists(RESULTS_LASE, "1gram")
hists_lase_2gram = get_lase_hists(RESULTS_LASE, "2gram")
hists_lase_3gram = get_lase_hists(RESULTS_LASE, "3gram")
hists_lase_4gram = get_lase_hists(RESULTS_LASE, "4gram")
hists_lase_5gram = get_lase_hists(RESULTS_LASE, "5gram")
hists_lase_6gram = get_lase_hists(RESULTS_LASE, "6gram")
hists_lase_7gram = get_lase_hists(RESULTS_LASE, "7gram")
hists_lase_8gram = get_lase_hists(RESULTS_LASE, "8gram")
hists_lase_9gram = get_lase_hists(RESULTS_LASE, "9gram")
hists_lase_10gram = get_lase_hists(RESULTS_LASE, "10gram")

In [194]:
# with context
hist_len_1gram_lase = get_lase_hists_len(RESULTS_LASE, "1gram")
hist_len_2gram_lase = get_lase_hists_len(RESULTS_LASE, "2gram")
hist_len_3gram_lase = get_lase_hists_len(RESULTS_LASE, "3gram")
hist_len_4gram_lase = get_lase_hists_len(RESULTS_LASE, "4gram")
hist_len_5gram_lase = get_lase_hists_len(RESULTS_LASE, "5gram")
hist_len_6gram_lase = get_lase_hists_len(RESULTS_LASE, "6gram")
hist_len_7gram_lase = get_lase_hists_len(RESULTS_LASE, "7gram")
hist_len_8gram_lase = get_lase_hists_len(RESULTS_LASE, "8gram")
hist_len_9gram_lase = get_lase_hists_len(RESULTS_LASE, "9gram")
hist_len_10gram_lase = get_lase_hists_len(RESULTS_LASE, "10gram")

423
1221
1735
2017
2174
2263
2317
2347
2363
2377


In [204]:
# without context
hist_len_1gram_lase = get_lase_hists_len(RESULTS_LASE, "1gram")
hist_len_2gram_lase = get_lase_hists_len(RESULTS_LASE, "2gram")
hist_len_3gram_lase = get_lase_hists_len(RESULTS_LASE, "3gram")
hist_len_4gram_lase = get_lase_hists_len(RESULTS_LASE, "4gram")
hist_len_5gram_lase = get_lase_hists_len(RESULTS_LASE, "5gram")
hist_len_6gram_lase = get_lase_hists_len(RESULTS_LASE, "6gram")
hist_len_7gram_lase = get_lase_hists_len(RESULTS_LASE, "7gram")
hist_len_8gram_lase = get_lase_hists_len(RESULTS_LASE, "8gram")
hist_len_9gram_lase = get_lase_hists_len(RESULTS_LASE, "9gram")
hist_len_10gram_lase = get_lase_hists_len(RESULTS_LASE, "10gram")

325
1058
1607
1934
2130
2240
2303
2337
2353
2368


In [205]:
hists_lase = []

hists_lase.append([]) # zero array
hists_lase.append(hists_lase_1gram)
hists_lase.append(hists_lase_2gram)
hists_lase.append(hists_lase_3gram)
hists_lase.append(hists_lase_4gram)
hists_lase.append(hists_lase_5gram)
hists_lase.append(hists_lase_6gram)
hists_lase.append(hists_lase_7gram)
hists_lase.append(hists_lase_8gram)
hists_lase.append(hists_lase_9gram)
hists_lase.append(hists_lase_10gram)


In [206]:
hists_lase_len = []

hists_lase_len.append([]) # zero array
hists_lase_len.append(hist_len_1gram_lase)
hists_lase_len.append(hist_len_2gram_lase)
hists_lase_len.append(hist_len_3gram_lase)
hists_lase_len.append(hist_len_4gram_lase)
hists_lase_len.append(hist_len_5gram_lase)
hists_lase_len.append(hist_len_6gram_lase)
hists_lase_len.append(hist_len_7gram_lase)
hists_lase_len.append(hist_len_8gram_lase)
hists_lase_len.append(hist_len_9gram_lase)
hists_lase_len.append(hist_len_10gram_lase)


### DBSCAN

In [148]:
for i in range(1, 11):
    hists = hists_lase[i]
    hists_len = hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=False)

        print(best_eps)

        tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                                   min_samples=2, 
                                                   metric='precomputed').fit(cur_dists)
        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


DBSCAN



Best eps: 0.06149425287356314
Best F-measure: 0.18754906921092465
0.06149425287356314
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 5
Number of outliers: 1/149 (0.671%)

Cohesion = 37.89767092344934
Separation = 2367.1412008881625
G1 = 362.0127455303709

Entropy = 3.944602
Purity = 0.175676
F-measure = 0.187549

Rand = 17.834%
Jaccard Index = 5.658%



DBSCAN



Best eps: 0.6005464480874317
Best F-measure: 0.9328815852305783
0.6005464480874317
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 4/149 (2.685%)

Cohesion = 49.49948347777997
Separation = 1730.2662643789113
G1 = 230.7760864177212

Entropy = 0.0
Purity = 1.0
F-measure = 0.932882

Rand = 99.282%
Jaccard Index = 85.902%



DBSCAN


...


Best eps: 0.9886363636363636
Best F-measure: 0.9059303485477981
0.9886363636363636
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 43.06110817398274
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



DBSCAN



Best eps: 0.9681857618512111
Best F-measure: 0.9059303485477981
0.9681857618512111
_______________________________
10 cos_distance
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 47.82635208852346
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



DBSCAN



Best eps: 0.993917725454889
Best F-measure: 0.8969818026865004
0.993917725454889
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 58.50957612773061
Separation = 5429.1372065148125
G1 = 740.7917342391871

Entropy = 0.0
Purity = 1.0
F-measure = 0.896982

Rand = 98.857%
Jaccard Index = 77.649%


In [143]:
hists = hists_lase[4]
hists_len = hists_lase_len[4]

cur_dists = np.array(get_dists(hists, pearsons_correlation_mean, hists_len))

min(cur_dists.flatten())
cur_dists

array([[0.        , 0.        , 0.65792061, ..., 1.01759491, 1.01759491,
        1.01759491],
       [0.        , 0.        , 0.65792061, ..., 1.01759491, 1.01759491,
        1.01759491],
       [0.65792061, 0.65792061, 0.        , ..., 1.0239148 , 1.0239148 ,
        1.0239148 ],
       ...,
       [1.01759491, 1.01759491, 1.0239148 , ..., 0.        , 0.        ,
        0.        ],
       [1.01759491, 1.01759491, 1.0239148 , ..., 0.        , 0.        ,
        0.        ],
       [1.01759491, 1.01759491, 1.0239148 , ..., 0.        , 0.        ,
        0.        ]])

### HAC average

In [200]:
for i in range(1, 11):
    hists = hists_lase[i]
    hists_len = hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'average'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link, with_step=0.005)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.03
Best F-measure: 0.5703295859730917
0.03
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 23
Number of outliers: 5/149 (3.356%)

Cohesion = 60.31539412669273
Separation = 14276.9871916422
G1 = 1477.6441713998092

Entropy = 1.18215
Purity = 0.590278
F-measure = 0.57033

Rand = 93.056%
Jaccard Index = 26.213%



Agglomerative Clustering



Best eps: 0.705
Best F-measure: 0.9352678641269244
0.705
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 26
Number of outliers: 3/149 (2.013%)

Cohesion = 47.73324417810008
Separation = 1773.5763188071564
G1 = 199.68717794822118

Entropy = 0.083321
Purity = 0.965753
F-measure = 0.935268

Rand = 99.301%
Jaccard Index = 86.738%



Agglomerative Clustering


...


Best eps: 0.995
Best F-measure: 0.9059303485477981
0.995
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 43.06110817398274
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



Agglomerative Clustering



Best eps: 0.985
Best F-measure: 0.9059303485477981
0.985
_______________________________
10 cos_distance
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 47.82635208852346
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



Agglomerative Clustering



Best eps: 0.995
Best F-measure: 0.8788609973173729
0.995
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 31
Number of outliers: 6/149 (4.027%)

Cohesion = 59.17319489948217
Separation = 5377.3618767029075
G1 = 773.5526816999444

Entropy = 0.0
Purity = 1.0
F-measure = 0.878861

Rand = 98.769%
Jaccard Index = 75.915%


In [198]:
for i in range(1, 11):
    hists = hists_lase[i]
    hists_len = hists_lase_len[i]
    

    dist = distances[3]
    dist_name = distances_names[3]

    cur_dists = get_dists(hists, dist, hists_len)
    link = 'average'
    best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                from_change_to_label, ids_per_label, 
                                agglomerative=True, linkage=link, with_step=0.005)

    print(best_eps)

    print("_______________________________")
    print(i, dist_name)
    print("_______________________________")

    tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                        affinity='precomputed', 
                                                        linkage=link, 
                                                        compute_full_tree=True,
                                                        distance_threshold=best_eps).fit(cur_dists)

    print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                            label_to_changes, from_change_to_label, ids_per_label,
                                  without_outliers=True)




Agglomerative Clustering



Best eps: 0.405
Best F-measure: 0.8916838567845277
0.405
_______________________________
1 pearsons_correlation_mean
_______________________________
Number of clusters: 29
Number of outliers: 4/149 (2.685%)

Cohesion = 64.94678548549366
Separation = 7177.122233137655
G1 = 715.5852519310482

Entropy = 0.069943
Purity = 0.972414
F-measure = 0.891684

Rand = 99.061%
Jaccard Index = 81.952%



Agglomerative Clustering



Best eps: 0.46
Best F-measure: 0.8995545096216235
0.46
_______________________________
2 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 61.68742077641115
Separation = 4515.652553922231
G1 = 526.0009573462897

Entropy = 0.0
Purity = 1.0
F-measure = 0.899555

Rand = 99.251%
Jaccard Index = 85.356%



Agglomerative Clustering


...


Best eps: 1.0
Best F-measure: 0.9059303485477981
1.0
_______________________________
8 pearsons_correlation_mean
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 58.971210853210906
Separation = 5299.696445572124
G1 = 613.2361491651543

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



Agglomerative Clustering



Best eps: 0.97
Best F-measure: 0.8788609973173729
0.97
_______________________________
9 pearsons_correlation_mean
_______________________________
Number of clusters: 31
Number of outliers: 6/149 (4.027%)

Cohesion = 59.51793788581148
Separation = 5305.168798665568
G1 = 744.6856529342381

Entropy = 0.0
Purity = 1.0
F-measure = 0.878861

Rand = 98.769%
Jaccard Index = 75.915%



Agglomerative Clustering



Best eps: 0.995
Best F-measure: 0.8788609973173729
0.995
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 31
Number of outliers: 6/149 (4.027%)

Cohesion = 59.17319489948217
Separation = 5377.3618767029075
G1 = 773.5526816999444

Entropy = 0.0
Purity = 1.0
F-measure = 0.878861

Rand = 98.769%
Jaccard Index = 75.915%


### HAC complete

In [149]:
for i in range(1, 11):
    hists = hists_lase[i]
    hists_len = hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'complete'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.25396825396825395
Best F-measure: 0.5510879027963193
0.25396825396825395
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 19
Number of outliers: 0/149 (0.0%)

Cohesion = 64.26128686741748
Separation = 14489.14902811083
G1 = 711.9665533803262

Entropy = 1.321847
Purity = 0.543624
F-measure = 0.551088

Rand = 92.708%
Jaccard Index = 25.07%



Agglomerative Clustering



Best eps: 0.6553987678987678
Best F-measure: 0.9328815852305783
0.6553987678987678
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 4/149 (2.685%)

Cohesion = 49.49948347777997
Separation = 1730.2662643789113
G1 = 230.7760864177212

Entropy = 0.0
Purity = 1.0
F-measure = 0.932882

Rand = 99.282%
Jaccard Index = 85.902%



Agglomerative Clustering


...


Best eps: 1.0
Best F-measure: 0.8878095431786706
1.0
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 43.68127240073953
Separation = 0.33343200763025743
G1 = 0.08506538335581805

Entropy = 0.0
Purity = 1.0
F-measure = 0.88781

Rand = 98.808%
Jaccard Index = 76.686%



Agglomerative Clustering



Best eps: 0.9717777495555132
Best F-measure: 0.8878095431786706
0.9717777495555132
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 48.540892566888026
Separation = 0.5734805019096936
G1 = 0.13284611306520083

Entropy = 0.0
Purity = 1.0
F-measure = 0.88781

Rand = 98.808%
Jaccard Index = 76.686%



Agglomerative Clustering



Best eps: 1.000841750692646
Best F-measure: 0.8788609973173729
1.000841750692646
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 31
Number of outliers: 6/149 (4.027%)

Cohesion = 59.17319489948217
Separation = 5377.3618767029075
G1 = 773.5526816999444

Entropy = 0.0
Purity = 1.0
F-measure = 0.878861

Rand = 98.769%
Jaccard Index = 75.915%


# LASE for fixed n (without context)

### DBSCAN

In [155]:
for i in range(1, 11):
    hists = hists_lase[i]
    hists_len = hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=False)

        print(best_eps)

        tufano_clustering = sklearn.cluster.DBSCAN(eps=best_eps, 
                                                   min_samples=2, 
                                                   metric='precomputed').fit(cur_dists)
        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


DBSCAN



Best eps: 0.08593073593073586
Best F-measure: 0.2216340614071988
0.08593073593073586
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 6
Number of outliers: 0/149 (0.0%)

Cohesion = 39.60702510537168
Separation = 2836.0744018749474
G1 = 397.64631655220364

Entropy = 3.753627
Purity = 0.208054
F-measure = 0.221634

Rand = 23.753%
Jaccard Index = 6.004%



DBSCAN



Best eps: 0.6005464480874317
Best F-measure: 0.94496212214333
0.6005464480874317
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 28
Number of outliers: 4/149 (2.685%)

Cohesion = 48.83444955205416
Separation = 1997.2847510863821
G1 = 250.72515737475507

Entropy = 0.0
Purity = 1.0
F-measure = 0.944962

Rand = 99.454%
Jaccard Index = 89.286%



DBSCAN


...


Best eps: 0.9886363636363636
Best F-measure: 0.9059303485477981
0.9886363636363636
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 43.09197905368005
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



DBSCAN



Best eps: 0.9681857618512111
Best F-measure: 0.9059303485477981
0.9681857618512111
_______________________________
10 cos_distance
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 47.90305922999988
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



DBSCAN



Best eps: 0.994017375492741
Best F-measure: 0.8969818026865004
0.994017375492741
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 58.58092141048598
Separation = 5419.291742417873
G1 = 739.6172380578655

Entropy = 0.0
Purity = 1.0
F-measure = 0.896982

Rand = 98.857%
Jaccard Index = 77.649%


### HAC average

In [209]:
for i in range(1, 11):
    hists = hists_lase[i]
    hists_len = hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'average'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link, with_step=0.005)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.035
Best F-measure: 0.5526319451806876
0.035
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 27
Number of outliers: 4/149 (2.685%)

Cohesion = 58.699828929147195
Separation = 14613.14806804667
G1 = 1591.6382427545284

Entropy = 1.13823
Purity = 0.613793
F-measure = 0.552632

Rand = 93.161%
Jaccard Index = 22.644%



Agglomerative Clustering



Best eps: 0.635
Best F-measure: 0.924721363647538
0.635
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 27
Number of outliers: 4/149 (2.685%)

Cohesion = 48.00401363599733
Separation = 1999.1812008376771
G1 = 239.07032011993098

Entropy = 0.061515
Purity = 0.972414
F-measure = 0.924721

Rand = 99.262%
Jaccard Index = 86.051%



Agglomerative Clustering


...


Best eps: 0.99
Best F-measure: 0.9059303485477981
0.99
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 43.09197905368005
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



Agglomerative Clustering



Best eps: 0.97
Best F-measure: 0.9059303485477981
0.97
_______________________________
10 cos_distance
_______________________________
Number of clusters: 29
Number of outliers: 6/149 (4.027%)

Cohesion = 47.90305922999988
Separation = 0.0
G1 = 0.0

Entropy = 0.0
Purity = 1.0
F-measure = 0.90593

Rand = 98.897%
Jaccard Index = 78.42%



Agglomerative Clustering



Best eps: 0.995
Best F-measure: 0.8788609973173729
0.995
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 31
Number of outliers: 6/149 (4.027%)

Cohesion = 59.229678896738776
Separation = 5371.354963246561
G1 = 772.6792512924173

Entropy = 0.0
Purity = 1.0
F-measure = 0.878861

Rand = 98.769%
Jaccard Index = 75.915%


### HAC complete

In [157]:
for i in range(1, 11):
    hists = hists_lase[i]
    hists_len = hists_lase_len[i]
    
    for j in range(len(distances)):
        dist = distances[j]
        dist_name = distances_names[j]
        
        cur_dists = get_dists(hists, dist, hists_len)
        link = 'complete'
        best_eps = find_eps_with_brute_force_universal(cur_dists, label_to_changes, 
                                    from_change_to_label, ids_per_label, 
                                    agglomerative=True, linkage=link)

        print(best_eps)

        print("_______________________________")
        print(i, dist_name)
        print("_______________________________")
        
        tufano_clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=None,
                                                            affinity='precomputed', 
                                                            linkage=link, 
                                                            compute_full_tree=True,
                                                            distance_threshold=best_eps).fit(cur_dists)

        print_clustering_results_tufano_unique(tufano_clustering, cur_dists,
                                label_to_changes, from_change_to_label, ids_per_label,
                                      without_outliers=True)
        
        


Agglomerative Clustering



Best eps: 0.25555555555555554
Best F-measure: 0.5773679592767631
0.25555555555555554
_______________________________
1 jaccard_metric
_______________________________
Number of clusters: 23
Number of outliers: 0/149 (0.0%)

Cohesion = 62.321974252005546
Separation = 15066.229912630779
G1 = 881.8877402018372

Entropy = 1.087539
Purity = 0.610738
F-measure = 0.577368

Rand = 93.978%
Jaccard Index = 25.893%



Agglomerative Clustering



Best eps: 0.6851851851851851
Best F-measure: 0.9300052669180184
0.6851851851851851
_______________________________
1 canberra_metric
_______________________________
Number of clusters: 29
Number of outliers: 3/149 (2.013%)

Cohesion = 49.20290456166456
Separation = 2065.7927233867895
G1 = 266.8154263400863

Entropy = 0.022227
Purity = 0.993151
F-measure = 0.930005

Rand = 99.235%
Jaccard Index = 84.944%



Agglomerative Clustering


...


Best eps: 1.0
Best F-measure: 0.8878095431786706
1.0
_______________________________
10 canberra_metric
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 43.72003687273956
Separation = 0.44376737044213965
G1 = 0.11274616818272076

Entropy = 0.0
Purity = 1.0
F-measure = 0.88781

Rand = 98.808%
Jaccard Index = 76.686%



Agglomerative Clustering



Best eps: 1.0
Best F-measure: 0.8878095431786706
1.0
_______________________________
10 cos_distance
_______________________________
Number of clusters: 30
Number of outliers: 6/149 (4.027%)

Cohesion = 48.604484709421094
Separation = 1.1333683674879762
G1 = 0.26163099291742953

Entropy = 0.0
Purity = 1.0
F-measure = 0.88781

Rand = 98.808%
Jaccard Index = 76.686%



Agglomerative Clustering



Best eps: 1.000844951264482
Best F-measure: 0.8788609973173729
1.000844951264482
_______________________________
10 pearsons_correlation_mean
_______________________________
Number of clusters: 31
Number of outliers: 6/149 (4.027%)

Cohesion = 59.229678896738776
Separation = 5371.354963246561
G1 = 772.6792512924173

Entropy = 0.0
Purity = 1.0
F-measure = 0.878861

Rand = 98.769%
Jaccard Index = 75.915%
